## Visualize MTBS and Existing Vegetation Cover
- [Landfire EVC (Existing Vegetation Cover)](https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Vegetation_EVC_v1_4_0)
- [ESA WorldCover 10m](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)
- [ESRI Landcover](https://gee-community-catalog.org/projects/S2TSLULC/#earth-engine-snippet)

In [21]:
import ee
import geemap
import json
from datetime import datetime

In [22]:
# EE authentication
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-milagrosbecerra245')

In [23]:
# Load datasets
burned_area = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")
landtype = ee.ImageCollection("LANDFIRE/Vegetation/EVC/v1_4_0")
landcover = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")
burned_severity = ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")

# Create a map object
Map = geemap.Map(center=[37.5, -120], zoom=6)

# Define visualization parameters
burned_vis_params = {
    'min': 0,
    'max': 3,
    'palette': ['white', 'orange', 'red', 'black']
}

colorbrewer_YlGnBu = [
    '#f7fcfd', '#e0ecf4', '#bfd3e6', '#9ebcda', '#8c96c6',
    '#8c6bb1', '#88419d', '#6e016b'
]

landcover_vis_params = {
    'bands': ['b1'],  
    'min': 1,
    'max': 9,
    'palette': [
        'FF0000', '00FF00', '0000FF', 'FFFF00', '00FFFF', 
        'FF00FF', 'FFFFFF', '808080', '000000'
    ]  
}

# Add layers to the map
Map.addLayer(burned_area, {}, "Burned Area Boundaries")
Map.addLayer(burned_severity.median(), burned_vis_params, "Burned Severity Mosaics")
Map.addLayer(landtype.mosaic(), {'min': 11, 'max': 356, 'palette': colorbrewer_YlGnBu}, "Existing Vegetation Cover")
Map.addLayer(landcover.median(), landcover_vis_params, "ESRI Global Landcover")

# Display the map with controls
Map.addLayerControl()
Map


Map(center=[37.5, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

## Retrieve Landsat 7 and ESA Landcover

In [24]:
# Add Earth Engine dataset
dem = ee.Image("USGS/SRTMGL1_003")
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select("landcover")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003").select(
    ["B1", "B2", "B3", "B4", "B5", "B7"]
)
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

# Add Earth Engine layers to Map
Map.addLayer(dem, vis_params, "SRTM DEM", True, 0.5)
Map.addLayer(landcover, {}, "Land cover")
Map.addLayer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2.0},
    "Landsat 7",
)
Map.addLayer(states, {}, "US States")

Map

Map(bottom=6649.0, center=[37.5, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

## Retrieve US Cities 

In [25]:
Map = geemap.Map()

In [26]:
url = "https://raw.githubusercontent.com/giswqs/leafmap/master/examples/data/us_cities.geojson"


In [28]:
Map.add_point_layer(url, popup=["name", "pop_max"], layer_name="US Cities")
Map

Map(bottom=6649.0, center=[37.5, -120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…